# Intro
**ModelTrainer** trains all the models found in the *model_type*'s json file (i.e. *cifar.json*)

In [34]:
import os, pdb, json, re, vis_utils
# Import codes to obtain requisite commands
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.models import Sequential, load_model, Model, model_from_json
from keras.layers import Dense, Flatten, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, Input
from keras.utils import np_utils, generic_utils
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
from tqdm import tqdm
import cv2

# File Loaders
1. **model_json**: This contains model details. The format involves a model set (i.e. *cifar*, etc) and relevant details such as image dimensions, training data size, etc

2. **top_level**: Lists the top level files (at this time, they are <span style="color:#006400">['jsons', 'activations', 'data', 'max_activations','history', 'tensorlogs', 'weights']</span>)

3. **dir_paths**: The relative directory paths to the **top_level** directories, i.e. *cifar/activations*

4. **jsons** : The list of *model.json* files, i.e. *cifar.json* within the **cifar/json** folder

5. **model_dirs**: The directory that each of the *model.json*'s training and activation data should reside, i.e. **weights/cifar_1/**

In [2]:
model_json = vis_utils.json_load('models.json')

In [3]:
model_type = 'cifar'
top_level= [_dir for _dir in os.listdir(model_type) if not _dir.startswith('.')]
dir_paths = {entry:os.path.join(model_type, entry) for entry in top_level}
jsons = os.listdir(dir_paths['jsons'])
model_dirs = [os.path.splitext(_item)[0] for _item in jsons]
train_data_dir = 'data/' + model_type + '/train'
test_data_dir = 'data/' + model_type + '/test'
class_name = {i:str(model_json[model_type]['labels'][i]) for i in range(len(model_json[model_type]['labels']))}

# Keras Image Data generator
Generate the batched keras image data files through **ImageDataGenerator**. Sets up the training and test data, using information from **model_json**

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen  = ImageDataGenerator(rescale=1./255)
batch_size = model_json[model_type]['batch']
#train_generator = train_datagen.flow_from_directory(
#        train_data_dir,
#        target_size=(model_json[model_type]['size']['width'],model_json[model_type]['size']['height']),
#        batch_size=batch_size,
#        classes=[str(_item) for _item in model_json[model_type]['labels']],
#        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(model_json[model_type]['size']['width'],model_json[model_type]['size']['height']),
        batch_size=batch_size,
        classes=[str(_item) for _item in model_json[model_type]['labels']],
        class_mode='categorical')

Found 10000 images belonging to 10 classes.


# <span style="color:red">Training</span>
Trains every model stored in *model.json* i.e. *cifar.json* - *cifar_1.json*, *cifar_2.json*, etc. The relevant data are stored in:
1. **history**: The model histories (Keras based) are stored in *model_type/history/model.csv* aka **cifar/history/cifar_1.csv**
2. **Tensorboard**: The Tensorboard data is stored in **cifar/tensorlogs/cifar_1/**
3. **Epoch Weights**: The epoch weights are stored in **cifar/weights/cifar_1/** as *weights-#.hdf5*
3. **Best Model**: The best model's weights are stored in **cifar/weights/cifar_1/** as *best.hdf5*

This sets up what is completed, so we can skip it in training

In [5]:
jsons[1]

'cifar_1.json'

In [78]:
if os.path.exists(os.path.join(dir_paths['supplement'], 'completed.txt')):
    _cont = open(os.path.join(dir_paths['supplement'], 'completed.txt'), 'r')
    dat = _cont.read().split()
    _cont.close()
else:
    dat = []
#for idx in range(len(jsons)):
idx = 1
# Setting up the directory structures
#if jsons[idx] in dat:
#    continue
print('Starting with ' + jsons[idx])
dir_list = vis_utils.dir_list_returner(dir_paths, ['weights','tensorlogs','history','supplement'], model_dirs[idx])
#Set up supplemental parameters
compile_params = vis_utils.json_load(os.path.join(dir_list['supplement'], 'compile.json'))
# Loading the Model file
with open(os.path.join(dir_paths['jsons'],jsons[idx]), 'r') as model_file:
    model = model_from_json(model_file.read())
weight_files = [_file for _file in os.listdir(dir_list['weights']) if _file.startswith('weights')]
weight_files.sort()
#model.load_weights(os.path.join(dir_list['weights'], weight_files[wt_idx]))    
#model.compile(loss = compile_params['loss'], optimizer = compile_params['optimizer'], metrics=compile_params['metrics'])
# Setting up the callbacks
#tensorboard_callback = TensorBoard(log_dir=dir_list['tensorlogs']+'/', histogram_freq=0, write_graph=True, write_images=False)
# Train
model.load_weights(os.path.join(dir_list['weights'], weight_files[14]))

Starting with cifar_1.json


In [79]:
#conv_1
kernels_to_keep = {"conv_1":[27,11,8,9,15,  8  ,  4 ,  16,   22 ,  18],
                   "conv_2":[12, 5, 1, 7, 10,0   , 1  ,  2   , 7  , 10],
                   "conv_3":[57, 60, 59, 61, 55, 10  ,  9  , 12 ,  14 ,  16],
                   "conv_4":[1,2,3,4,5, 1  ,  7  , 20  , 30 ,  36],
                   "conv_5":[0, 20, 27, 36, 43, 50  ,  9  ,  8  , 24 ,  46],
                   "conv_6":[53, 97,82, 25, 33, 117 , 113 , 110 , 104 ,  46],}

c_layers = ['conv_' + str(i) for i in range(1,7)]

for layer in c_layers:
    extract_weights = model.get_layer(layer).get_weights()

    for i in range(extract_weights[0].shape[3]):
        if i not in kernels_to_keep[layer]:
            extract_weights[0][:,:,:,i] = np.zeros(extract_weights[0][:,:,:,i].shape)
            extract_weights[1][i] = 0.0
    model.get_layer(layer).set_weights(extract_weights)

In [87]:
#dog_files = os.listdir(os.path.join(test_data_dir, 'dog'));
spec = 'automobile'
dog_files = os.listdir(os.path.join(test_data_dir, spec));
#os.path.exists(os.path.join(test_data_dir+'/dog', dog_files[4]))
final=[]
results = []
for _file in tqdm(dog_files):
    img = np.array([cv2.imread(os.path.join(test_data_dir+'/' +spec, _file))/255.])
    res = model.predict(img)
    results.append(res)
    final.append(int(np.where(res == np.max(res))[1]))

100%|██████████| 1000/1000 [00:03<00:00, 260.74it/s]


In [88]:
np.where(np.array(final)==3)[0].shape

(646,)

In [89]:
final

[3,
 3,
 3,
 4,
 4,
 3,
 3,
 3,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 4,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 4,
 3,
 4,
 4,
 3,
 3,
 3,
 4,
 3,
 3,
 4,
 3,
 3,
 3,
 4,
 3,
 4,
 3,
 3,
 3,
 3,
 4,
 3,
 4,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 3,
 3,
 3,
 4,
 3,
 4,
 4,
 3,
 3,
 4,
 4,
 4,
 3,
 4,
 3,
 3,
 4,
 4,
 4,
 4,
 3,
 4,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 4,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 4,
 4,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 4,
 4,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 4,
 3,
 4,
 3,
 3,
 4,
 3,
 3,
 3,
 4,
 3,
 4,
 4,
 3,
 3,
 4,
 4,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 3,
 3,
 4,
 3,
 4,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 3,
 4,
 4,
 3,
 3,
 4,
 4,
 3,
 3,
 3,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 3,
 3,
 4,
 3,
 4,
 3,
 3,
 3,
 3,
 3,
 4,
 3,
 4,
 4,
 3,
 3,
 3,
 4,
 3,
 3,
 4,
 4,
 4,
 3,


In [82]:
class_name

{0: 'airplane',
 1: 'automobile',
 2: 'bird',
 3: 'cat',
 4: 'deer',
 5: 'dog',
 6: 'frog',
 7: 'horse',
 8: 'ship',
 9: 'truck'}

In [ ]:
model_hist = model.fit_generator(
        train_generator,
        steps_per_epoch=model_json[model_type]['train']/model_json[model_type]['batch'],
        epochs = model_json[model_type]['epochs'],
        validation_data = test_generator,
        validation_steps = model_json[model_type]['test']/model_json[model_type]['batch'],
        verbose = 1,
        callbacks=[tensorboard_callback, checkpoint_callback, best_callback])
with open(os.path.join(dir_paths['history'], jsons[idx]), 'w') as _compile:
        json.dump(model_hist.history, _compile)
print('Completed with ' + jsons[idx])
with open(os.path.join(dir_paths['supplement'], 'completed.txt'), 'a') as _compile:
        _compile.write(jsons[idx]+'\n')